# Vaccinations Administered by County

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json
from scipy import stats
from api_keys import geoapify_key
from api_keys import google_api_key

In [2]:
# Load vaccine data into DataFrame
vaccine_df = pd.read_csv("../Keyana/COVID-19_Vaccinations_in_the_United_States_County.csv")
vaccine_df.head()

,Recip_County,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_65Plus,Series_Complete_5Plus,Series_Complete_12Plus,Series_Complete_18Plus,Series_Complete_65Plus
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0
4,Saginaw County,NaN,NaN,NaN,NaN,103054.0,101354.0,95769.0,32377.0


In [3]:
# Rename columns
vaccine_df = vaccine_df.rename(columns = {'Recip_County': 'County',
        'Administered_Dose1_Recip_5Plus': 'Administered Dose 1 (5 Plus)',
        'Administered_Dose1_Recip_12Plus': 'Administered Dose 1 (12 Plus)',
        'Administered_Dose1_Recip_18Plus': 'Administered Dose 1 (18 Plus)',
        'Administered_Dose1_Recip_65Plus': 'Administered Dose 1 (65 Plus)',
        'Series_Complete_5Plus': 'Administered Series (5 Plus)',
        'Series_Complete_12Plus': 'Administered Series (12 Plus)',
        'Series_Complete_18Plus': 'Administered Series (18 Plus)',
        'Series_Complete_65Plus': 'Administered Series (65 Plus)'})

vaccine_df

,County,Administered Dose 1 (5 Plus),Administered Dose 1 (12 Plus),Administered Dose 1 (18 Plus),Administered Dose 1 (65 Plus),Administered Series (5 Plus),Administered Series (12 Plus),Administered Series (18 Plus),Administered Series (65 Plus)
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0
4,Saginaw County,NaN,NaN,NaN,NaN,103054.0,101354.0,95769.0,32377.0
...,...,...,...,...,...,...,...,...,...
1962776,Adams County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1962777,Harrison County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1962778,Matagorda County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0
1962779,Pleasants County,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0


In [4]:
# Drop NaN
vaccine_df.dropna()

,County,Administered Dose 1 (5 Plus),Administered Dose 1 (12 Plus),Administered Dose 1 (18 Plus),Administered Dose 1 (65 Plus),Administered Series (5 Plus),Administered Series (12 Plus),Administered Series (18 Plus),Administered Series (65 Plus)
0,Washburn County,11097.0,10863.0,10368.0,4749.0,10311.0,10105.0,9663.0,4483.0
1,Taylor County,3145.0,3079.0,2966.0,1249.0,2949.0,2887.0,2792.0,1211.0
2,Nassau County,1384503.0,1329779.0,1232671.0,313054.0,1174924.0,1127179.0,1041792.0,268167.0
3,Lampasas County,11660.0,11468.0,10849.0,3920.0,9999.0,9845.0,9330.0,3448.0
5,Echols County,1572.0,1537.0,1458.0,366.0,1273.0,1245.0,1182.0,319.0
...,...,...,...,...,...,...,...,...,...
781419,Northumberland County,57909.0,57337.0,55068.0,19835.0,47678.0,47407.0,45483.0,15854.0
781420,Brazoria County,226767.0,221577.0,201780.0,41601.0,200238.0,198221.0,180766.0,38109.0
781421,Broward County,1534351.0,1509940.0,1407121.0,353489.0,1288295.0,1279787.0,1194457.0,297576.0
781422,Norfolk city,140790.0,137273.0,130028.0,25330.0,123259.0,121927.0,115749.0,22792.0


In [ ]:
# Add columns for totals 
vaccine_df[['Administered Dose 1 (5 Plus)', 'Administered Dose 1 (12 Plus)',
            'Administered Dose 1 (18 Plus)', 'Administered Dose 1 (65 Plus)']].agg('sum', axis=1)

vaccine_df[['Administered Series (5 Plus)', 
    
]] = vaccine_df[] +
    vaccine_df['Administered Series (12 Plus)'] + vaccine_df['Administered Series (18 Plus)'] + 
    vaccine_df['Administered Series (65 Plus)']

vaccine_df

# Total Dose 1 Administered
#            'Total Series Administered']
# for index, rox in vaccine_df.iterrows():
#     sum_dose1 = vaccine_df.iloc[1:4].sum()
#     sum_series = vaccine_df.iloc[5:8].sum()


In [ ]:
#Add dose 1 columns together for total dose 1 administered
dose1_5plus = df('
                 ').sum()
print(dose1_5plus)
# 12plus_dose1 = 
# 18plus_dose1 = 
# 65plus_dose1 =
# total_dose1 = df({'column1': ['administered_dose1_recip_5plus', 'administered_dose1_recip_12plus', 
#                               'administered_dose1_recip_18plus', 'administered_dose1_recip_65plus'], 
#                   'column2': ['series_complete_5plus', 'series_complete_12plus', 'series_complete_18plus', 
#                              'series_complete_65plus']})
# total_dose1['sum'] = df['column1'] + df['column2']

#Add series columns together for total series administrated
#total_series = 


In [ ]:
# Response
response_county = requests.get(url_county).json()
print(json.dumps(response_county, indent=4, sort_keys=True))

In [ ]:
# API for lat and lon
county = []
url_county = f"https://maps.googleapis.com/maps/api/geocode/json?address={county}&key={google_api_key}"



# Find latitude and longitude for each county
countys = [len("Recip_County")]

# List to hold lat and lon data
lat = []
lon = []

for county in countys:
    response = requests.get(url_county + county).json()
    lat.append["results"]["location"]["lat"]
    lon.append["results"]["location"]["lng"]
    


In [11]:
# Statistical summary for total 1 dose  (need to change)
#Alter once totals for vaccinations of dose 1 and series calculated
mean_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_5 = vaccine_df['Administered Dose 1 (5 Plus)'].groupby(vaccine_df['County']).sem()

mean_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_12 = vaccine_df['Administered Dose 1 (12 Plus)'].groupby(vaccine_df['County']).sem()

mean_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_18 = vaccine_df['Administered Dose 1 (18 Plus)'].groupby(vaccine_df['County']).sem()

mean_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).mean()
median_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).median()
variance_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).var()
std_dev_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).std()
sem_dose1_65 = vaccine_df['Administered Dose 1 (65 Plus)'].groupby(vaccine_df['County']).sem()

summary_dose1_df = pd.DataFrame({"Mean Dose 1 Vaccine Distribution (Age 5+)": mean_dose1_5,
                           "Mean Dose 1 Vaccine Distribution (Age 12+)": mean_dose1_12,
                           "Mean Dose 1 Vaccine Distribution (Age 18+)": mean_dose1_18,
                           "Mean Dose 1 Vaccine Distribution (Age 65+)": mean_dose1_65,
                           "Median Dose 1 Vaccine Distribution (Age 5+)": median_dose1_5,
                           "Median Dose 1 Vaccine Distribution (Age 12+)": median_dose1_12,
                           "Median Dose 1 Vaccine Distribution (Age 18+)": median_dose1_18,
                           "Median Dose 1 Vaccine Distribution (Age 65+)": median_dose1_65,
                           "Dose 1 Vaccine Distribution Variance (Age 5+)": variance_dose1_5,
                           "Dose 1 Vaccine Distribution Variance (Age 12+)": variance_dose1_12,
                           "Dose 1 Vaccine Distribution Variance (Age 18+)": variance_dose1_18,
                           "Dose 1 Vaccine Distribution Variance (Age 65+)": variance_dose1_65,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 5+)": std_dev_dose1_5,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 12+)": std_dev_dose1_12,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 18+)": std_dev_dose1_18,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 65+)": std_dev_dose1_65,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 5+)": sem_dose1_5,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 12+)": sem_dose1_12,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 18+)": sem_dose1_18,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 65+)": sem_dose1_65})
summary_dose1_df

,Mean Dose 1 Vaccine Distribution (Age 5+),Mean Dose 1 Vaccine Distribution (Age 12+),Mean Dose 1 Vaccine Distribution (Age 18+),Mean Dose 1 Vaccine Distribution (Age 65+),Median Dose 1 Vaccine Distribution (Age 5+),Median Dose 1 Vaccine Distribution (Age 12+),Median Dose 1 Vaccine Distribution (Age 18+),Median Dose 1 Vaccine Distribution (Age 65+),Dose 1 Vaccine Distribution Variance (Age 5+),Dose 1 Vaccine Distribution Variance (Age 12+),Dose 1 Vaccine Distribution Variance (Age 18+),Dose 1 Vaccine Distribution Variance (Age 65+),Dose 1 Vaccine Distribution Std. Dev (Age 5+),Dose 1 Vaccine Distribution Std. Dev (Age 12+),Dose 1 Vaccine Distribution Std. Dev (Age 18+),Dose 1 Vaccine Distribution Std. Dev (Age 65+),Dose 1 Vaccine Distribution Std. Err. (Age 5+),Dose 1 Vaccine Distribution Std. Err. (Age 12+),Dose 1 Vaccine Distribution Std. Err. (Age 18+),Dose 1 Vaccine Distribution Std. Err. (Age 65+)
County,,,,,,,,,,,,,,,,,,,,
Abbeville County,10912.168067,8282.409699,8011.433110,3320.132107,10968.0,9579.5,9225.5,3668.5,1.094135e+05,9.873651e+06,8.873254e+06,1.024692e+06,330.777114,3142.236589,2978.800828,1012.270719,21.441096,128.495610,121.812224,41.394828
Acadia Parish,36399.697479,25923.887960,24852.961538,8039.775920,36603.5,32126.0,30617.0,9288.5,6.380612e+05,1.389618e+08,1.220543e+08,6.757946e+06,798.787346,11788.207867,11047.818125,2599.604975,51.777693,482.055668,451.778880,106.305753
Accomack County,26423.117647,18847.535117,18028.471572,6628.777592,26564.0,22302.5,21222.5,7447.0,7.553205e+05,6.788174e+07,5.932309e+07,5.431652e+06,869.091773,8239.037738,7702.148397,2330.590443,56.334852,336.919308,314.964270,95.304931
Ada County,345584.073684,330898.605263,232091.906355,61511.364548,344287.5,329666.5,269429.5,68891.0,3.469113e+07,2.765795e+07,8.572762e+09,4.428517e+08,5889.918049,5259.082868,92589.208692,21044.042042,427.299566,381.533972,3786.254308,860.554874
Adair County,9300.009454,6688.456522,6398.343227,2065.986622,9447.5,6874.0,6648.0,2256.5,1.333444e+07,1.611145e+07,1.440862e+07,9.726400e+05,3651.635695,4013.906516,3795.868276,986.225114,118.350193,82.070422,77.612299,20.164872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yukon-Koyukuk Census Area,4265.495798,3143.862876,2888.528428,682.623746,4344.5,3412.5,3082.5,718.0,2.826143e+04,1.087676e+06,8.329019e+05,3.395072e+04,168.111369,1042.916974,912.634588,184.257223,10.897042,42.648047,37.320404,7.534838
Yuma County,98025.907563,64241.140468,59062.545987,15720.854515,81897.0,4848.0,4597.5,1506.0,8.904918e+09,5.610406e+09,4.624824e+09,2.891964e+08,94365.873068,74902.641371,68006.061085,17005.775568,4325.252701,2165.865797,1966.446029,491.734697
Zapata County,9877.840336,4478.801003,3932.983278,843.782609,9927.0,0.0,0.0,0.0,4.451051e+05,2.222667e+07,1.713848e+07,7.887194e+05,667.161940,4714.517051,4139.864443,888.098756,43.245685,192.790939,169.291646,36.317059


In [ ]:
# Statistical summary for series
mean_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).mean()
median_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).median()
variance_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).std()
sem_series_5 = vaccine_df['Administered Series (5 Plus)'].groupby(vaccine_df['County']).sem()

mean_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).mean()
median_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).median()
variance_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).std()
sem_series_12 = vaccine_df['Administered Series (12 Plus)'].groupby(vaccine_df['County']).sem()

mean_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).mean()
median_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).median()
variance_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).std()
sem_series_18 = vaccine_df['Administered Series (18 Plus)'].groupby(vaccine_df['County']).sem()

mean_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).mean()
median_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).median()
variance_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).var()
std_dev_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).std()
sem_series_65 = vaccine_df['Administered Series (65 Plus)'].groupby(vaccine_df['County']).sem()

summary_series_df = pd.DataFrame({"Mean Series Vaccine Distribution (Age 5+)": mean_series_5,
                                  "Mean Series Vaccine Distribution (Age 12+)": mean_series_12,
                                  "Mean Series Vaccine Distribution (Age 18+)": mean_series_18,
                                  "Mean Series Vaccine Distribution (Age 65+)": mean_series_65,
                                  "Median Series Vaccine Distribution (Age 5+)": median_series_5,
                                  "Median Series Vaccine Distribution (Age 12+)": median_series_12,
                                  "Median Series Vaccine Distribution (Age 18+)": median_series_18,
                                  "Median Series Vaccine Distribution (Age 65+)": median_series_65,
                                  "Series Vaccine Distribution Variance (Age 5+)": variance_series_5,
                                  "Series Vaccine Distribution Variance (Age 12+)": variance_series_12,
                                  "Series Vaccine Distribution Variance (Age 18+)": variance_series_18,
                                  "Series Vaccine Distribution Variance (Age 65+)": variance_series_65,
                                  "Series Vaccine Distribution Std. Dev (Age 5+)": std_dev_series_5,
                                  "Series Vaccine Distribution Std. Dev (Age 12+)": std_dev_series_12,
                                  "Series Vaccine Distribution Std. Dev (Age 18+)": std_dev_series_18,
                                  "Series Vaccine Distribution Std. Dev (Age 65+)": std_dev_series_65,
                                  "Series Vaccine Distribution Std. Err (Age 5+)": sem_series_5,
                                  "Series Vaccine Distribution Std. Err (Age 12+)": sem_series_12,
                                  "Series Vaccine Distribution Std. Err (Age 18+)": sem_series_18,
                                  "Series Vaccine Distribution Std. Err (Age 65+)": sem_series_65})
summary_series_df

In [ ]:
# Configure map
map_plot_1 = airports_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Number of Vaccines Administrated",
    scale = 0.5, 
    color = "County"
)

# Display the map plot
map_plot_1